In [1]:
from risk_targeted_hazard import *  # not yet a pypi package, must be installed locally


### prep query from NSHM

In [ ]:
hazard_id = 'NSHM_v1.0.4'

hf_name = 'test.hdf5'
all_sites = False   # returns list of 214 named and 3600 gridded sites
named_sites = True  # only returns named sites--returns all 214 if site_list=None or the subset included in site_list, see below
short_list = False  # a small subset of vs30, imt, agg. see below.

# sites of interest
if all_sites:
    sites = pd.concat([create_sites_df(),create_sites_df(named_sites=False)])
elif named_sites:
    site_list = None # ['Auckland','Wellington']
    site_list = ['Auckland','Blenheim','Christchurch','Dunedin','Gisborne','Greymouth','Masterton','Napier','Nelson',
                'Queenstown','Tauranga','Wellington']
    sites = create_sites_df(named_sites=named_sites,site_list=site_list)
else:
    cropped_grid = False
    grid_limits = [-38,np.inf,-np.inf,176] # [min_lat,max_lat,min_lon,max_lon]
    sites = create_sites_df(named_sites=named_sites,cropped_grid=cropped_grid,grid_limits=grid_limits)

# lists for vs30, imt, and aggregation statistics
if short_list:
    vs30_list = [275,375]
    imt_list = ['SA(0.5)', 'SA(1.5)']
    agg_list = ["mean","0.1","0.5","0.9"]
    
else:
    vs30_list = [150, 175, 225, 250, 275, 375, 400, 525, 750]
    imt_list = ['PGA', 'SA(0.1)', 'SA(0.15)', 'SA(0.2)', 'SA(0.25)', 'SA(0.3)', 'SA(0.35)', 'SA(0.4)', 'SA(0.5)', 'SA(0.6)', 'SA(0.7)', 'SA(0.8)', 'SA(0.9)', 'SA(1.0)', 'SA(1.25)', 'SA(1.5)', 'SA(1.75)', 'SA(2.0)', 'SA(2.5)', 'SA(3.0)', 'SA(3.5)', 'SA(4.0)', 'SA(4.5)', 'SA(5.0)', 'SA(6.0)','SA(7.5)', 'SA(10.0)']
    agg_list = ["mean","0.1","0.5","0.9"]
    

# call a location to get the imtls that are returned
res = next(get_hazard_curves(sites['latlon'][:1], vs30_list[:1], [hazard_id], imt_list[:1], agg_list[:1]))
imtl_list = [float(val.lvl) for val in res.values]


### create the hdf5

In [ ]:
# create a temporary dictionary
data = {}

# prep metadata
imtls = {}
for imt in imt_list:
    imtls[imt] = imtl_list

data['metadata'] = {}
data['metadata']['quantiles'] = [float(q) for q in agg_list[1:]]
data['metadata']['acc_imtls'] = imtls
data['metadata']['disp_imtls'] = convert_imtls_to_disp(imtls)
data['metadata']['sites'] = sites
data['metadata']['vs30s'] = vs30_list

# query the nshm
data['hcurves'] = {}
print('Querying hazard curves...')
data['hcurves']['hcurves_stats'] = query_nshm_hcurves(hazard_id, sites, vs30_list, imt_list, imtl_list, agg_list)


In [ ]:
add_uniform_hazard_spectra(data,hazard_rps=np.array([25,50,100,250,500,1000,2500]))
save_hdf(hf_name, data)

### add risk spectra

In [ ]:
R_assumptions = {}

####
baseline_rp = 500
baseline_risk = 1.5e-5
cmr = 4
####

beta = 0.45
design_point = stats.lognorm(beta,scale=cmr).cdf(1)
p_fatality_given_collapse = 0.1

R_assumptions['baseline_risk'] = baseline_risk
R_assumptions['R_rps'] = [500,1000,2500]
for R_rp in R_assumptions['R_rps']:
    risk_factor = baseline_rp/R_rp
    fatality_risk_target = round(baseline_risk*risk_factor,12)
    R_assumptions[f'APoE: 1/{R_rp}'] = {'risk_factor': risk_factor,'fatality_risk_target': fatality_risk_target}

R_assumptions


risk_assumptions = {}
for R_rp in R_assumptions['R_rps']:
    key = f'APoE: 1/{R_rp}'
    risk_assumptions[key] = { 'risk_factor':R_assumptions[f'APoE: 1/{R_rp}']['risk_factor'],
                              'fatality_risk_target':R_assumptions[f'APoE: 1/{R_rp}']['fatality_risk_target'],
                              'R_rp':R_rp,
                              'ls_risk_target': round(R_assumptions[f'APoE: 1/{R_rp}']['fatality_risk_target'] / p_fatality_given_collapse,12),
                              'cmr':cmr,
                              'beta':beta,
                              'design_point':design_point,
                              'p_fatality_given_collapse':p_fatality_given_collapse
                               }
    
risk_assumptions

In [ ]:
# get risk values
print('Calculating risk informed values.')
data['risk_design'] = {}
intensity_type = 'acc'
data['risk_design'][intensity_type] = {}
data['risk_design'][intensity_type]['risk_assumptions'] = risk_assumptions
data['risk_design'][intensity_type]['im_risk'] = calculate_risk_design_intensities(data,risk_assumptions)

In [ ]:
update_hdf_for_risk(hf_name,data)